In [1]:
!pip install transformers
!pip install torch
!pip install tensorflow
!pip install tf-keras
!pip install gensim
!pip install spacy
!pip install nltk
!pip install boto3 pandas
!pip install pyarrow
!pip install fastparquet
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 22.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.6/786.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 20.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 22.1 MB/s eta 0:00:0000:0100:01


In [2]:
from transformers import pipeline

# Cargar el modelo de resumen
summarizer = pipeline('summarization', model='robercg33/distilbart-cnn-12-6-finetuned')

# Definir la función para resumir el texto
def split_text(text, chunk_size=1000):
    """Divide el texto en fragmentos más pequeños."""
    for i in range(0, len(text), chunk_size):
        yield text[i:i + chunk_size]

def summarize_text(text: str) -> str:
    summaries = []
    
    # Divide el texto en fragmentos y resume cada fragmento
    for chunk in split_text(text, chunk_size=1024):  # Ajusta el chunk_size según tu modelo
        summary = summarizer(chunk, max_length=150, min_length=30, do_sample=False)  # Valores más razonables
        summarized_text = summary[0]['summary_text']
        summaries.append(summarized_text)
    
    # Une todos los resúmenes de los fragmentos
    return " ".join(summaries)

# Uso de la función
texto = "Aquí va el texto largo que deseas resumir."
resumen = summarize_text(texto)
print(resumen)

2024-09-08 21:14:45.409722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-08 21:14:45.426489: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-08 21:14:45.431701: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-08 21:14:46.580950: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


config.json:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 150, but your input_length is only 17. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


El texto largo que deseas resumir. Aquívuela en un ella lloa, un telenamiento, se púbela en una lente. También, una muyuyo lloa hay hay hay años que se fit fit and cost prohibata.


In [3]:
import re
import string
import json
import pickle
import multiprocessing
from gensim import corpora
from gensim.models import LdaMulticore
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

# Download NLTK resources if needed
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

topics_dict = {
    0: "Education",
    1: "Conflict",
    2: "Politics",
    3: "Justice",
    4: "Economy",
    5: "Incidents",
    6: "Global"
}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [4]:
class LDAModelWrapper:
    def __init__(self, lda_model, dictionary, config):
        """
        Initializes the LDAModelWrapper with a trained LDA model, dictionary, and configuration.

        Parameters:
        - lda_model: A trained Gensim LdaMulticore model.
        - dictionary: A Gensim Dictionary object used for the model.
        - config: Configuration dictionary or JSON loaded parameters.
        """
        self.lda_model = lda_model
        self.dictionary = dictionary
        self.config = config
        self.stemming = config.get('stemming', False)
        self.preprocess_library = config.get('preprocess_library', 'nltk')
        self.no_above = config.get('no_above', 0.5)
        self.no_below = config.get('no_below', 5)
        self.topics_dict = config.get('topics_dict', {})  # Load the topic labels dictionary from config


    def preprocess_text(self, text):
        """
        Preprocesses the input text based on configuration settings.

        Parameters:
        - text: A string representing the text to preprocess.

        Returns:
        - A list of processed tokens.
        """
        # Convert text to lowercase
        text = text.lower()

        # Remove punctuation and numbers
        text = re.sub(r'[\d]', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))

        if self.preprocess_library == "nltk":
            # Tokenize the text
            tokens = word_tokenize(text)

            # Remove stopwords using NLTK
            stop_words_nltk = set(stopwords.words('english'))
            tokens = [word for word in tokens if word not in stop_words_nltk]

            # Apply lemmatization
            lemmatizer = WordNetLemmatizer()
            tokens = [lemmatizer.lemmatize(word) for word in tokens]

        elif self.preprocess_library == "spacy":
            # Process the text with spaCy
            doc = nlp(text)

            # Filter and lemmatize tokens
            tokens = [token.lemma_ for token in doc if token.text.lower() not in STOP_WORDS and not token.is_punct and not token.is_space]
        else:
            # If invalid preprocess library, raise an error
            raise ValueError("Invalid preprocess library, must be -> ['nltk', 'spacy']")

        # Apply stemming if required
        if self.stemming:
            stemmer = SnowballStemmer("english")
            tokens = [stemmer.stem(word) for word in tokens]

        return tokens

    def get_topic_scores(self, text, threshold=0.1, verbose=False):
        """
        Gets the topic distribution and scores for an unseen document.

        Parameters:
        - text: The unseen document as a string.
        - threshold: Minimum score threshold to display topics.

        Returns:
        - A list of tuples containing topic indices and their scores.
        """
        # Preprocess the unseen document
        tokens = self.preprocess_text(text)
        bow_vector = self.dictionary.doc2bow(tokens)

        # Get topic distribution for the document
        topic_scores = sorted(self.lda_model[bow_vector], key=lambda tup: -tup[1])

        # Filter topics based on the threshold and map to human-readable labels
        filtered_topics = [(self.topics_dict.get(index, f"Topic {index}"), score)
                           for index, score in topic_scores if score >= threshold]

        # Print the topics and scores IF verbose flag added
        #if verbose:
        #  for label, score in filtered_topics:
        #      print(f"Score: {score}\nTopic: {label}\n")

        return filtered_topics

    def save_model(self, filepath):
        """
        Saves the LDA model and dictionary to a file.

        Parameters:
        - filepath: The path to save the model file.
        """
        with open(filepath, 'wb') as file:
            pickle.dump({'lda_model': self.lda_model, 'dictionary': self.dictionary, 'config': self.config}, file)
        print(f"Model saved to {filepath}")

    @classmethod
    def load_model(cls, filepath):
        """
        Loads the LDA model and dictionary from a file.

        Parameters:
        - filepath: The path to the model file to load.

        Returns:
        - An instance of LDAModelWrapper.
        """
        with open(filepath, 'rb') as file:
            data = pickle.load(file)
        print(f"Model loaded from {filepath}")
        return cls(data['lda_model'], data['dictionary'], data['config'])

In [5]:
import pickle
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer

# Asegúrate de que estas dependencias estén cargadas, o instala las que falten:
# pip install nltk
# pip install spacy
# python -m spacy download en_core_web_sm (si usas spaCy)

# Primero, carga el modelo LDA desde "lda_model.pkl"
lda_wrapper = LDAModelWrapper.load_model('lda_model.pkl')

# Crear una función que procese un texto y lo pase por el modelo
def process_text_and_get_topics(text, lda_wrapper, threshold=0.1, verbose=False):
    """
    Procesa un texto y obtiene los temas principales usando el modelo LDA.

    Parameters:
    - text: El texto que deseas analizar.
    - lda_wrapper: Instancia del LDAModelWrapper que contiene el modelo y diccionario.
    - threshold: El umbral para filtrar los temas con puntuación baja.
    - verbose: Si es True, imprime los temas y sus puntuaciones.

    Returns:
    - Lista de temas y sus puntuaciones.
    """
    # Obtener los temas usando el método de la clase
    topics = lda_wrapper.get_topic_scores(text, threshold, verbose)
    
    return topics

# Ejemplo de uso: pasa el texto al modelo y obtiene los temas
texto = "Este es un ejemplo de texto que quiero analizar con el modelo LDA."
temas = process_text_and_get_topics(texto, lda_wrapper, threshold=0.1, verbose=True)

# Mostrar los temas y sus puntuaciones
print(temas)


Model loaded from lda_model.pkl
[('Education', 0.58583146), ('Conflict', 0.20156047), ('Politics', 0.14002243)]


In [6]:
texto = "En las últimas horas, Canal 13 emitió el primer spot de su nuevo reality show “Palabra de Honor: ¿Lealtad o traición?”. El video, difundido por TV y redes sociales, muestra a actores personificados como militares que marchan y entonan melodías castrenses. La locación elegida fue el campus central de la Universidad de Santiago de Chile (Usach). El uso de referencias militares en el campus generó un gran malestar en la comunidad universitaria, dado que el recinto fue bombardeado el 11 de septiembre de 1973. Desde Usach calificaron el spot como un “ataque profundo a la memoria institucional”. El gobierno universitario expresó su descontento y solicitó la renuncia de Daniela Benincasa, directora de la Corporación Cultural de la Usach, responsable de los acuerdos de arriendo del campus para la grabación de comerciales y spots."
resumen = summarize_text(texto)
result = process_text_and_get_topics(resumen, lda_wrapper, threshold=0.1, verbose=True)
print(result)

[('Education', 0.62289655), ('Politics', 0.16163136), ('Conflict', 0.15856084)]


In [7]:
import boto3
import pandas as pd
import io
import csv

# Configura el acceso a AWS S3
s3 = boto3.client('s3')

input_bucket_name = 'complu-bucket'
input_folder_key = 'new-news'
output_bucket_name = 'complu-bucket'
output_file_key = 'categories/news_categories.csv'

# Función para obtener el archivo más reciente de un bucket S3
def get_latest_parquet(bucket_name, folder_key):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_key)
    all_files = response.get('Contents', [])
    
    # Filtra solo los archivos CSV
    csv_files = [file for file in all_files if file['Key'].endswith('.parquet')]
    
    # Ordena los archivos por fecha de modificación
    latest_file = max(csv_files, key=lambda x: x['LastModified'])
    
    return latest_file['Key']

# Función para leer un archivo CSV desde S3
def read_csv_from_s3(bucket_name, file_key):
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    return pd.read_csv(io.BytesIO(obj['Body'].read()))

# Función para escribir un archivo CSV a S3
def write_csv_to_s3(dataframe, bucket_name, file_key):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket=bucket_name, Key=file_key, Body=csv_buffer.getvalue())


# Función para eliminar el archivo si existe
def delete_file_from_s3(bucket_name, file_key):
    try:
        s3.delete_object(Bucket=bucket_name, Key=file_key)
        print(f"Archivo {file_key} eliminado del bucket {bucket_name}.")
    except Exception as e:
        print(f"Error al eliminar el archivo {file_key}: {e}")



In [17]:
def summarize_text_news():
    processed_df = read_csv_from_s3(output_bucket_name, output_file_key)

    # Encuentra el top 5 por cada categoría
    top_summaries = processed_df.groupby('category').apply(lambda x: x.nlargest(5, 'points')).reset_index(drop=True)

    # Lista para guardar los resúmenes finales
    final_output_rows = []

    print("---------------------------------")
    print("---------------------------------")
    print("---------------------------------")
    print("the summarization begins")
    # Ejecuta `summarize_text` para los top 5 por cada categoría
    for idx, row in top_summaries.iterrows():
        title = row['title']  # Extrae el título
        summary = row['summary']
        category = row['category']
        points = row['points']
        
        # print(row)
        # Ejecuta el resumen solo para los top 5
        resumen = summarize_text(summary)  # Aquí ejecutas el resumen

        # Guarda el resultado final con el resumen
        final_output_rows.append({
            'id': idx,
            'summary': resumen,   # Resumen procesado
            'title': title,
            'category': category,  # Categoría original
            'points': points       # Puntos originales
        })
        print(str(idx)+": "+title)

    # Crea un nuevo DataFrame con los resultados finales
    final_output_df = pd.DataFrame(final_output_rows)

    # Guarda el nuevo archivo con los resúmenes finales
    final_output_file_key = "summaries/news_summaries.csv" # Puedes ajustar el nombre del archivo de salida
    delete_file_from_s3(output_bucket_name, final_output_file_key)
    write_csv_to_s3(final_output_df, output_bucket_name, final_output_file_key)

    print(f"Archivo final con resúmenes guardado en: s3://{output_bucket_name}/{final_output_file_key}")

In [18]:
import boto3
import pandas as pd
from io import BytesIO

def categorized_news():
    print("---------------------------------")
    print("---------------------------------")
    print("---------------------------------")
    print("the categorization begins")
    s3 = boto3.client('s3', 
                    aws_access_key_id='XXXXXXXXXXXXXXXXX',
                    aws_secret_access_key='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
                    region_name='eu-north-1')
    # Busca el archivo más reciente en el bucket
    latest_file_key = get_latest_parquet(input_bucket_name, input_folder_key)
    print(latest_file_key)

    # Lee el archivo Parquet
    response = s3.get_object(Bucket=input_bucket_name, Key=latest_file_key)
    parquet_data = response['Body'].read()
    df = pd.read_parquet(BytesIO(parquet_data))

    # Procesa la columna 'summary' y guarda los resultados
    output_rows = []

    for idx, row in df.iterrows():  # Itera sobre las filas del DataFrame
        title = row['title']  # Extrae el título
        summary = row['body']  # Extrae el resumen (cuerpo)
        #if idx >= 5:
        #    break 
        #print(len(summary))
        #if len(summary) > 5000:
        #    print(f"El texto es demasiado largo ({len(summary)} caracteres), se truncará a 6999 caracteres.")
        #    summary = summary[:5000] 
        #else:
        texto = summary
        #resumen = summarize_text(texto)  # Asume que tienes esta función definida
        result = process_text_and_get_topics(texto, lda_wrapper, threshold=0.1, verbose=True)
        
        category, points = result[0]  # La primera tupla (categoría y valor)
            
            # Guarda el resumen, categoría y valor numérico en la lista
        output_rows.append({
            'title': title,
            'summary': summary,
            'category': category,  # String de la primera tupla
            'points': points,       # Valor numérico de la primera tupla
        })
        print(str(idx)+": "+title)

    # Crea un nuevo DataFrame con los resultados
    output_df = pd.DataFrame(output_rows)

    # Guarda el DataFrame en un CSV y lo sube al bucket S3
    delete_file_from_s3(output_bucket_name, output_file_key)
    delete_file_from_s3(input_bucket_name, input_folder_key)
    write_csv_to_s3(output_df, output_bucket_name, output_file_key)

    print(f"Archivo procesado y guardado en: s3://{output_bucket_name}/{output_file_key}")
    summarize_text_news()


In [ ]:
import time
import os
from datetime import datetime, timedelta

def clear_console():
    if os.name == 'nt':  # Windows
        os.system('cls')
    else:  # Unix/Linux/Mac
        os.system('clear')

# Intervalo de tiempo en segundos (15 minutos)
interval_minutes = 15
interval_seconds = interval_minutes * 60

s3 = boto3.client('s3', 
                    aws_access_key_id='XXXXXXXXXXXXXXXXXXXXXXXXXXXX',
                    aws_secret_access_key='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
                    region_name='eu-north-1')

while True:
    # Marca el tiempo de inicio del ciclo
    start_time = datetime.now()
    
    # Limpia la consola
    clear_console()
    
    # Llama a las funciones
    categorized_news()
    
    # Calcula el tiempo de finalización esperado del ciclo
    end_time = start_time + timedelta(seconds=interval_seconds)
    
    # Espera hasta el final del intervalo de tiempo
    now = datetime.now()
    time_to_sleep = (end_time - now).total_seconds()
    
    if time_to_sleep > 0:
        print(f"Esperando {interval_minutes} minutos antes de la próxima ejecución...")
        time.sleep(time_to_sleep)
    else:
        print("El procesamiento tomó más tiempo del esperado.")


---------------------------------
---------------------------------
---------------------------------
the categorization begins
new-news/news_20240711191500_to_20240712000000.parquet
0: The Killers' 'Mr. Brightside' Just Broke Two Guinness World Records | Radio 94.5
1: 'ALF' Actor Benji Gregory’s Poignant Last Posts Before Death at 46
2: World population is projected to grow from 8.2 billion to a peak of 10.3 billion in 2080s, UN says – Winnipeg Free Press
3: China stabbing: Last Iowan hurt headed back to United States
4: Biden to hold 'big boy' news conference 
5: Accusations of US siding with UAE in Sudan war are misleading 
6: Minnesota cop helps thwart plot to kidnap, rape and assassinate British TV personality – Orange County Register
7: Virginia: Dominion plans a $500 million plus LNG facility
8: Leura author Rick Asensio publishes magical children's book | Blue Mountains Gazette | Katoomba, NSW
9: Wisconsin GOP candidate TIED with Dem senator who votes with Biden 95% of the time